In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

In [5]:
proxy = None
chrome_options = Options()

if proxy:
    proxy_options = {
        "proxy": {
            "https": f"http://{proxy}",
        }
    }
else:
    proxy_options = None

# Additional options for undetected Chrome
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--use-subprocess")

driver = uc.Chrome(options=chrome_options, seleniumwire_options=proxy_options)
chrome_pid = driver.service.process.pid

In [6]:
driver.get("https://fix-price.com/catalog/knigi/p-4000504-seriya-uchebnyh-posobiy-lomonosovskaya-shkola-eksmo")

In [7]:
from scrapy.http import HtmlResponse

response = HtmlResponse(driver.current_url, body=driver.page_source, encoding="utf-8", request=None)
all_urls = response.xpath("//a/@href").extract()

In [8]:
all_urls

['https://fix-price.com/doc/cookie_policy',
 '/stores',
 '/work/store',
 '/specials',
 '/bonus',
 '/',
 '/catalog',
 '/catalog/skoro-v-shkolu',
 '/catalog/pochti-darom',
 '/catalog/kollektsiya-eritreya',
 '/catalog/kollektsiya-bambino',
 '/catalog/dlya-zdorovykh-nachinaniy',
 '/catalog/letnee-nastroenie',
 '/catalog/sad-i-ogorod',
 '/catalog/spets-tsena-po-karte',
 '/catalog/novinki',
 '/catalog/seychas-pokupayut',
 '/catalog/produkty-i-napitki',
 '/catalog/bytovaya-khimiya',
 '/catalog/igrushki',
 '/catalog/kosmetika-i-gigiena',
 '/catalog/krasota-i-zdorove',
 '/catalog/dlya-doma',
 '/catalog/dekor-dlya-doma-tovary-dlya-prazdnika',
 '/catalog/suveniry-i-podarki',
 '/catalog/kantstovary',
 '/catalog/knigi',
 '/catalog/avto-moto-velo',
 '/catalog/odezhda',
 '/catalog/aksessuary',
 '/catalog/sport-i-otdykh',
 '/catalog/dlya-zhivotnykh',
 '/catalog/meditsinskie-tovary',
 '/catalog/prazdnik-kruglyy-god',
 '/catalog/skoro-v-shkolu',
 '/catalog/skoro-v-shkolu/tetradi-bloknoty-dnevniki',
 '/c

In [9]:
from mvodolagin_personal_imports import *

2024-08-27 00:37:02.285 | INFO     | mvodolagin_personal_imports:<module>:122 - Dotenv loading status: True


In [23]:
from bs4 import BeautifulSoup, Comment

html_dir = r"C:\WorkData\TextTailor\scraped_data\fixprice_spider\raw_data_run0"

all_urls = []
not_found = []

for html_file in Path(html_dir).glob("*.html"):
    with open(html_file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    meta_url = soup.find('meta', property="og:url")
    if meta_url and meta_url.get('content'):
        all_urls.append(meta_url['content'])
    else:
        not_found.append(html_file)


In [28]:
already_scraped_fp = Path(r"C:\WorkData\TextTailor\scraped_data\fixprice_spider\already_scraped.json")

with open(already_scraped_fp, "w", encoding="utf-8") as f:
    json.dump(all_urls, f, ensure_ascii=False, indent=4)
    

In [24]:
not_found

[WindowsPath('C:/WorkData/TextTailor/scraped_data/fixprice_spider/raw_data_run0/item_698.html')]

In [19]:
def find_original_url(soup):
    urls = []

    # Check in meta tags
    meta_url = soup.find('meta', property="og:url")
    if meta_url and meta_url.get('content'):
        urls.append(meta_url['content'])
        print(1)

    # Check in canonical link
    canonical_link = soup.find('link', rel="canonical")
    if canonical_link and canonical_link.get('href'):
        urls.append(canonical_link['href'])
        print(2)

    # Check in HTML comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        if "URL" in comment:
            urls.append(comment.strip())

    # Check in JavaScript
    scripts = soup.find_all('script')
    for script in scripts:
        if script.string and 'url' in script.string.lower():
            urls.append(script.string.strip())

    return urls

In [20]:
find_original_url(soup)[0]

1


'https://fix-price.com/catalog/dlya-doma/p-5012654-lampa-nastolnaya-akkumulyatornaya-s-kabelem---v-assortimente'